In [2]:
from collections import defaultdict
from scipy.stats import itemfreq
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import feature
from PIL import Image as IMG
import numpy as np
import pandas as pd 
import operator
import cv2
import os 
import shutil

from IPython.core.display import HTML 
from IPython.display import Image

In [ ]:
features = pd.DataFrame()
features['image'] = imgs

In [7]:
def color_analysis(img):
    # obtain the color palatte of the image 
    palatte = defaultdict(int)
    for pixel in img.getdata():
        palatte[pixel] += 1
    
    # sort the colors present in the image 
    sorted_x = sorted(palatte.items(), key=operator.itemgetter(1), reverse = True)
    light_shade, dark_shade, shade_count, pixel_limit = 0, 0, 0, 25
    for i, x in enumerate(sorted_x[:pixel_limit]):
        if all(xx <= 20 for xx in x[0][:3]): ## dull : too much darkness 
            dark_shade += x[1]
        if all(xx >= 240 for xx in x[0][:3]): ## bright : too much whiteness 
            light_shade += x[1]
        shade_count += x[1] 
    light_percent = round((float(light_shade)/shade_count)*100, 2)
    dark_percent = round((float(dark_shade)/shade_count)*100, 2)
    return light_percent, dark_percent

In [8]:
def perform_color_analysis(img,flag):
    #path = image path
    im = IMG.open(path) 

    size = im.size
    halves = (size[0]/2, size[1]/2)
    im1 = im.crop((0, 0, size[0], halves[1]))
    im2 = im.crop((0, halves[1], size[0], size[1]))
    try:
        
        light_percent1, dark_percent1 = color_analysis(im1)

        light_percent2, dark_percent2 = color_analysis(im2)
    except Exception as e:
        return None

    light_percent = (light_percent1 + light_percent2)/2 
    dark_percent = (dark_percent1 + dark_percent2)/2 
    print(dark_percent,light_percent)
    if flag == 'black':

        return dark_percent
    elif flag == 'white':
        return light_percent
    else:
        return None

In [11]:
features['dullness'] = features['image'].apply(lambda x : perform_color_analysis(x,'black'))
features['whiteness'] = features['image'].apply(lambda x : perform_color_analysis(x,'white'))

0.0 0.0
7.185 50.0
95.76 0.0
0.0 9.755
0.0 0.0
0.0 0.0
2.215 0.0
0.0 6.08
0.0 0.0
1.77 0.0
0.0 1.765
7.98 0.0
31.895 0.0
0.0 12.035
0.0 2.415
1.715 0.0
2.71 0.0
49.620000000000005 0.0
0.0 0.0
0.0 0.0
18.825 0.0
0.0 0.0
100.0 0.0
0.0 26.29
35.205 0.0
0.0 0.0
5.81 9.105
0.0 0.0
0.0 1.125
0.0 26.24
0.0 35.46
0.0 0.0
0.0 0.0
0.0 0.0
57.22 0.0
20.315 4.87
0.0 0.0
0.0 4.23
0.0 3.74
18.5 0.0
0.0 11.62
0.0 3.36
20.985 0.0
23.69 22.89
1.685 0.0
0.0 0.0
0.0 0.0
5.415 0.0
0.0 10.8
0.0 0.0
0.0 21.71
0.0 41.19
0.0 0.0
0.0 0.0
64.27000000000001 0.0
0.0 80.385
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
9.465 0.0
32.28 0.0
47.28 2.225
0.0 39.04
0.0 0.0
0.0 0.0
52.30500000000001 0.0
0.0 3.07
4.955 0.0
9.235 0.0
0.0 45.975
0.0 0.0
12.9 0.0
0.0 0.0
13.145 0.0
16.215 0.0
0.0 2.82
0.0 86.02
4.845 0.0
0.0 38.74
0.0 4.8
0.0 0.0
6.645 0.0
0.0 0.0
5.325 16.22
35.495 2.36
76.21000000000001 0.0
20.494999999999997 0.0
0.0 0.0
71.445 5.51
0.0 9.01
22.44 0.0
0.0 0.0
0.0 6.15
0.0 82.045
0.0 0.0
0.0 0.0
11.31500

In [13]:
def getSize(filename):
    filename = images_path + filename
    st = os.stat(filename)
    return st.st_size

def getDimensions(filename):
    filename = images_path + filename
    img_size = IMG.open(filename).size
    return img_size 

In [14]:
features['image_size'] = features['image'].apply(getSize)
features['temp_size'] = features['image'].apply(getDimensions)
features['width'] = features['temp_size'].apply(lambda x : x[0])
features['height'] = features['temp_size'].apply(lambda x : x[1])


KeyError: "['average_color'] not found in axis"

In [15]:
features = features.drop(['temp_size'], axis=1)

In [17]:
def get_blurrness_score(image):
    path =  images_path + image 
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(image, cv2.CV_64F).var()
    return fm

In [18]:
features['blurrness'] = features['image'].apply(get_blurrness_score)

In [20]:
def final_data_token(x):
    print(x)
    a = x.split('@')
    return a[0]
def final_data_imagenumber(x):
    a = x.split('@')
    return a[1]
features['token'] = features.image.apply(final_data_token)
features['image_num'] = features.image.apply(final_data_imagenumber)
features.drop('image',axis = 1,inplace=True)

QYnhyaf2@5.jpg
QYdyuYWc@2.jpg
QY7mnqTN@3.jpg
QYU2Ks7E@0.jpg
QYPeNEFq@2.jpg
QYHO5BfH@12.jpg
QYH_y9n9@4.jpg
QYT3GOay@0.jpg
QYQW4hlo@4.jpg
QYY-pnFB@6.jpg
QYHDlVPk@2.jpg
QY0uHGS9@10.jpg
QYgC160L@0.jpg
QYcBDf4Y@4.jpg
QYYjnwUY@1.jpg
QYg4saG3@0.jpg
QYp6eisI@10.jpg
QYav6bUA@3.jpg
QYbiVxc_@6.jpg
QYk5xOVl@3.jpg
QYiz5rqt@1.jpg
QY0m7LQ4@1.jpg
QYSJ9WOR@8.jpg
QYaGnlMv@6.jpg
QYAHkLTY@0.jpg
QY9t1wrC@8.jpg
QYQeMpIY@1.jpg
QYxOr2gE@0.jpg
QYSO9kYN@12.jpg
QYveKGwE@0.jpg
QYJaIsSI@8.jpg
QY3WQqD5@1.jpg
QYtDE4Ih@4.jpg
QYRmT2bn@1.jpg
QYMj1i9a@1.jpg
QY1pXO_s@0.jpg
QYx2Jyuh@2.jpg
QYU-eqKv@0.jpg
QYoKGPXz@4.jpg
AYiSzOdU@6.jpg
QYo1W3fV@6.jpg
QYmizIvB@6.jpg
QYGaPltp@11.jpg
QYnt9b5P@4.jpg
QYrKy6Pt@5.jpg
QYAuh2iF@2.jpg
QYIfHU-b@1.jpg
QYjuJcr9@2.jpg
QYOW1ye6@3.jpg
QY89-dS-@11.jpg
QYx6cplw@0.jpg
QY1dAau_@0.jpg
QYr5zQHm@2.jpg
QYlmdaW2@5.jpg
QYlmMBjw@1.jpg
QYQWqkbf@3.jpg
QY2hL0EI@8.jpg
QYVVpHoC@1.jpg
QYvexzkq@5.jpg
QYUVSFJh@7.jpg
QYjlfZqI@5.jpg
QYq1euzf@9.jpg
QY3ZLu5z@6.jpg
QYJlFldv@0.jpg
QYwtxlFF@2.jpg
QYHSLXer@0.jpg
QYsm